
# Om å finne illustrasjoner i bøker fra NB digital

Fra en oversikt over alle illustrasjoner i NB-digital (ca. 480 000 bøker) kan vi studere selve illustrasjonene. Hva som kan vises er avhengig av tilgangsnivå, og hvor vi er, på jobb på NB eller på privatmaskin. 

Det frie materialet kan nås fra hvor som helst, mens det som har restriksjoner også er begrenset innad i NB. For begrenset materiale er det kun mulig å få ut deler av en illustrasjon, men det er mulig å mekke det til så man får sett alt. Foreløpig er ikke det gjort. 

### Litt startkode

In [13]:
import dhlab.module_update as mu
mu.update("nbpictures", silent = True)
from nbpictures import show_illustrations_urn, urns_from_super, load_picture, get_urls_from_illustration_data, get_illustration_data_from_book
import nbpictures as nbp
from IPython.display import HTML, Markdown, display
from skimage import data, io, filters, feature, color
import pandas as pd
import matplotlib.pyplot as plt
import os

Hent ut noen URN-er, eller bruk URN-er funnet annet sted

In [15]:
import imp

In [16]:
imp.reload(nbp)

<module 'nbpictures' from 'C:\\Users\\yoons\\Documents\\GitHub\\illustrations_bokhylla\\nbpictures.py'>

In [11]:
small_scale = 0.59
large_scale = 1.58

In [19]:
def get_urls_from_illustration_data(illus, part = True, scale = None, cuts = True, cutout = None, delta = 0):
    """part sets size of output of page, if part is True it returns a cut out of image
    using position data in illus is a dictionary of with entries and values like this: 
    {'height': 270, 'hpos': 251, 'page': 'digibok_2017081626006_0018', 'resolution': 400, 'vpos': 791, 'width': 373} 
    the variable cuts, if true allows cropping of image - restricted images must not go over 1024 x 1024 pixels
    
    The variable cutout, if different from zero, picks out a part of the picture, specifically made for
    finding the signature part of illustrations.
    """
    
    urn = "URN:NBN:no-nb_" + illus['page']
    
    if scale == None:
        if int(illus['resolution']) >= 300 or int(illus['resolution']) < 100:
            scale = large_scale
        else:
            scale = small_scale
            
    height = int(illus['height']) + 2*delta
    width = int(illus['width']) + 2*delta
    vpos = int(illus['vpos']) - delta
    hpos = int(illus['hpos']) - delta
    
    # check cutout, then that value will take precedence
    if cutout != None:
        (widthc, heightc) = cutout
        vpos = vpos + width - widthc
        hpos = hpos + height - heightc
        width = widthc
        height = heightc
        
    if cuts != False:
        if width * scale > 1024:
            width = int(1024/scale)
        if height * scale > 1024:
            height = int(1024/scale)
            

    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(width * scale), 
            height = int(height * scale), 
            vpos = int(vpos * scale), 
            hpos = int(hpos * scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

In [51]:
#!/usr/bin/env python
# coding: utf-8

# In[8]:


import sqlite3
import os
import tarfile
import shutil
import dhlab.nbtokenizer as tok


# In[4]:


def alto_extract(altofile, to_path = '.'):
    """
    Pakk ut tarfil til ramdisk og returnerer mappen filene ligger i
    """


    tf = tarfile.open(altofile, 'r')
    # lag mappe på disk med navnet foran det som står foran .tar
    filename = os.path.basename(altofile)
    dname = filename.split('.tar')[0]
    ndir = os.path.join(to_path, dname )
    os.mkdir(ndir)
    # Pakk ut alt til mappen
    tf.extractall(ndir)
    tf.close()
    return ndir


def process_alto(ndir):
    """Hent ut alle ordene i alto-filen, og legg til paragraf og sidenummer. Mappen ndir peker til mappen der alto-xml-filene ligger"""
    
    import xml.etree.ElementTree as ET
    import shutil
    
    # XML-filene ligger i mappen ndir, så gå gjennom med os.walk()
    # Alle filene blir liggende i variabelen f
    r,d,f = next(os.walk(ndir))
    
    # hent sidene i teksten og legg dem i variabelen pages
    # skip metadatafilene - tekstene har sidenummer representert som 4-sifrede nummer, f.eks. 0014
    pages = []
    
    for page in f:
        pag = page.split('.xml')[0].split('_')[-1]
        try:
            int(pag)
            pages.append((page, int(pag)))
        except:
            True
        
    # Gå gjennom side for side og hent ut teksten. Delte ord blir lagt i variabelen hyph,
    # teksten i text. Alle ord får et sekvensnummer relativt til boka det står i, samtidig
    # som alle avsnitt blir nummerert fortløpende
    
    para_num = 1
    word_num = 1

    text = []
    hyph = []

    hyp1 = ""
    hyp2 = ""
    
    # sorter variabelen pages på sidenummer, andre ledd i tuplet
    for page in sorted(pages, key=lambda x: x[1]):
        page_file = os.path.join(r, page[0])
        page_num = page[1]
        
        # parse XML-fila og få tak i rotelementet root
        tree = ET.parse(page_file)
        root = tree.getroot()
        
        # Gå gjennom XML-strukturen via TextBlock, som er avsnittselementet
        for paragraph in root.findall(".//TextBlock"):
            
            # Finn alle ordene i avsnittet, som attributter til elementet String,
            # og sjekk om det foreligger en orddeling -
            # i så fall ligger hele ordet i attributtet SUBS_CONTENT, mens første ledd av orddelingen
            # ligger i CONTENT. Om det ikke er noen orddeling ligger ordet i attributtet CONTENT.
            # Burde fungere også med orddelinger over sideskift
            
            # Ordet lagres sammen med sekvensnummeret og sekvensnummeret for avsnittet står i,
            # i tillegg til sidenummeret, som kan være greit for oppslag i bokhylla, i forbindelse
            # med generering av konkordanser.
            
            for string in paragraph.findall(".//String"):
                if 'SUBS_TYPE' in string.attrib:
                    if string.attrib['SUBS_TYPE'] == "HypPart1":
                        #tokens = tok.tokenize(string.attrib['SUBS_CONTENT'])
                        tokens = [string.attrib['SUBS_CONTENT']]
                        for token in tokens:
                            text.append((token, word_num, para_num, page_num))
                            word_num += 1
                    elif string.attrib['SUBS_TYPE'] == "HypPart2":
                        hyp2 = string.attrib['CONTENT']
                        hyph.append((hyp1, hyp2))
                else:
                    #tokens = tok.tokenize(string.attrib['CONTENT'])
                    tokens = [string.attrib['CONTENT']]
                    for token in tokens:
                        text.append((token, word_num, para_num, page_num))
                        word_num += 1
            para_num += 1
    # returner teksten som en sekvens av tupler, sammen med orddelingene, også som en sekvens av tupler
    return text #, hyph

In [65]:
def extract_image_alto(ndir):
    """Hent ut alle bildene i alto-filen. Mappen ndir peker til mappen der tarfilene ligger"""
    
    import xml.etree.ElementTree as ET
    import shutil
    
    ns = {'mix': "http://www.loc.gov/mix/"} 
    # XML-filene ligger i mappen ndir, så gå gjennom med os.walk()
    # Alle filene blir liggende i variabelen f
    
    r,d,f = next(os.walk(ndir))
    
    
    # hent oppløsningen fra metsfil
    metsfile = [fmets for fmets in f if fmets.endswith('mets.xml')]
    if metsfile != []:
        metsfile = os.path.join(r, metsfile[0])
        tree = ET.parse(metsfile)
        root = tree.getroot()
    
        resolutions = {element.text for element in root.findall(".//mix:XphysScanResolution", ns)}
        resolution = list(resolutions)[0]
    else:
        resolution = 100 
    
    pages = []
    
    for page in f:
        pag = page.split('.xml')[0].split('_')
        try:
        #    int(pag[2])
            pages.append((page, pag[2]))
            
        except:
            #print(pag)
            True
    # print(pages)
    # alle avsnitt blir nummerert fortløpende
    
    illustrations = []
    # sorter variabelen pages på sidenummer, andre ledd i tuplet
    for page in sorted(pages, key=lambda x: x[1]):
        page_file = os.path.join(r, page[0])
        page_num = page[1]
        file_identifier = page[0].split('.')[0] # remove .xml
        #print(page[0], page[1])
        # parse XML-fila og få tak i rotelementet root
        tree = ET.parse(page_file)
        root = tree.getroot()
        #print(root)
        # Gå gjennom XML-strukturen via TextBlock, som er avsnittselementet
        for paragraph in root.findall(".//*[@TYPE='Illustration']"):
            illustrations.append(
                {'page': file_identifier, 
                 'pagenum':page_num,
                 'resolution': resolution,
                 'hpos': paragraph.attrib["HPOS"],
                 'vpos':paragraph.attrib["VPOS"],
                 'width':paragraph.attrib["WIDTH"],
                 'height':paragraph.attrib["HEIGHT"],
                 'type': 'Illustration'
                 }
            )
        for paragraph in root.findall(".//*[@TYPE='GraphicalElement']"):
            illustrations.append(
                {'page': file_identifier,
                 'pagenum':page_num,
                 'resolution': resolution,
                 'hpos': paragraph.attrib["HPOS"],
                 'vpos':paragraph.attrib["VPOS"],
                 'width':paragraph.attrib["WIDTH"],
                 'height':paragraph.attrib["HEIGHT"],
                 'type': 'GraphicalElement'
                 }
            )
        
    return illustrations


In [22]:
from nbpictures import iiif_manifest

In [107]:
def display_books(books, width = 100):
    """A dictionary of urns - urls is displayed """
    import sys
    html_wrapper = lambda x: """<style>
    img {{
        width:{mxw}px;
        height:auto;
        max-widht:}}
    </style>
    <body>{body}</body>""".format(body = x, mxw = width)

    div_wrapper = lambda x: """<div>{div_content}</div>""".format(div_content = x)
    book_divs = ""
    for u in books:
        try:
            mf = iiif_manifest(u)
            #print('iif')
            thumbnail = "<img src = '{thumbnail}'></img>".format(thumbnail = mf['thumbnail']['@id'])
            #print('thumbnail')
            metainfo =  '\n'.join(["<b>{label}</b>{val}".format(label = x['label'], val = x['value']) for x in mf['metadata']])        
            imgs = '\n'.join(["<img src='{img_http}'></img>".format(img_http = pic_url) for pic_url in books[u]])
            #print('images')
            book_divs += div_wrapper(thumbnail + metainfo + imgs)
            #print('book_divs')
        except KeyboardInterrupt:
            break;
        except:
            print("Unexpected error:", sys.exc_info()[0])
            print("Problemer med: ", u, e)
            True
    return html_wrapper(book_divs)

In [ ]:
def markdown_books(books, width = 100):
    """A dictionary of urns - urls is displayed """
    
    markdown_wrapper = lambda x: """
    <body>{body}</body>""".format(body = x)

    div_wrapper = lambda x: """<div>{div_content}</div>""".format(div_content = x)
    
    book_divs = ""
    for u in books:
        try:
            mf = iiif_manifest(u)
            thumbnail = "<h3>Forside</h3> <img src='{thumbnail}'></img> <h3>Metadata</h3>".format(thumbnail = mf['thumbnail']['@id'])
            metainfo =  '\n'.join([" <b>{label}</b> {val}".format(label = x['label'], val = x['value']) for x in mf['metadata']])        
            imgs = '\n'.join(["<img style='float:right' src='{img_http}' width = {width}></img>".format(img_http = pic_url, width = width) for pic_url in books[u]])
            book_divs += div_wrapper(thumbnail + metainfo + imgs)
        except:
            True
    return book_divs

In [41]:
def markdown_illustrations(books, width = 100):
    """A list of resolver urls is displayed """
    import re
    
    markdown_wrapper = lambda x: """
    <body>{body}</body>""".format(body = x)

    div_wrapper = lambda x: """<div>{div_content}</div>""".format(div_content = x)
    book_divs = ""
    for u in books:
        urn = re.findall("URN:NBN:no-nb_digibok_[0-9]{13}", u)[0]
        mf = iiif_manifest(urn)
        thumbnail = "<h3>Forside</h3> <img src='{thumbnail}'></img> <h3>Metadata</h3>".format(thumbnail = mf['thumbnail']['@id'])
        metainfo =  '\n'.join([" <b>{label}</b> {val}".format(label = x['label'], val = x['value']) for x in mf['metadata']])        
        imgs = "<img style='float:right' src='{img_http}' width = {width}></img>".format(img_http = u, width = width) 
        book_divs += div_wrapper(thumbnail + metainfo + imgs)
    return book_divs

In [111]:
def dict_sample( a_dict, n = 10):
    from random import sample
    keys = list(a_dict.keys())
    sample_dict = {u:a_dict[u] for u in sample(keys, min(n, len(keys)))}
    return sample_dict

In [45]:
r, d, f = next(os.walk('../../skillingsviser-master/skillingsviser_alto/'))


In [80]:
illustrations = {"URN:NBN:no-nb_" + '_'.join(alto.split('_')[:2]):extract_image_alto(os.path.join(r,alto)) for alto in d}

#illustrations = [x for y in illustrations for x in y]

#illustrations

In [81]:
illustrations

{'URN:NBN:no-nb_digibok_2017103026032': [],
 'URN:NBN:no-nb_digibok_2018021528001': [{'page': 'digibok_2018021528001_I1',
   'pagenum': 'I1',
   'resolution': '400',
   'hpos': '67',
   'vpos': '168',
   'width': '138',
   'height': '177',
   'type': 'Illustration'}],
 'URN:NBN:no-nb_digibok_2018021528002': [{'page': 'digibok_2018021528002_C1',
   'pagenum': 'C1',
   'resolution': '400',
   'hpos': '72',
   'vpos': '1036',
   'width': '1161',
   'height': '778',
   'type': 'Illustration'},
  {'page': 'digibok_2018021528002_I3',
   'pagenum': 'I3',
   'resolution': '400',
   'hpos': '104',
   'vpos': '277',
   'width': '136',
   'height': '159',
   'type': 'Illustration'}],
 'URN:NBN:no-nb_digibok_2018021528003': [{'page': 'digibok_2018021528003_I1',
   'pagenum': 'I1',
   'resolution': '400',
   'hpos': '96',
   'vpos': '255',
   'width': '112',
   'height': '133',
   'type': 'Illustration'}],
 'URN:NBN:no-nb_digibok_2018021528004': [{'page': 'digibok_2018021528004_C1',
   'pagenum': '

Konverter bøker til illustrasjonsdata, formatet er en dict av

```{urn: [url, url, ..],
urn2: [url, url, ..],
...
}```

In [158]:
skillingsbilder = {urn:[get_urls_from_illustration_data(illus, part=True, cuts=False) for illus in illustrations[urn]] for urn in illustrations}

In [159]:
Markdown('\n\n'.join([ u + " https://www.nb.no/items/" + u + " \n\n" + '\n\n'.join(skillingsbilder[u]) for u in skillingsbilder]))

URN:NBN:no-nb_digibok_2017103026032 https://www.nb.no/items/URN:NBN:no-nb_digibok_2017103026032 



URN:NBN:no-nb_digibok_2018021528001 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528001 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528001_I1/105,265,218,279/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528002_C1/113,1636,1834,1229/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528002_I3/164,437,214,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528003_I1/151,402,176,210/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528004_C1/268,352,1554,1540/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528006_C1/188,537,1434,1325/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528006_I1/135,447,218,274/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528008 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528008_I1/86,211,472,396/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528012 



URN:NBN:no-nb_digibok_2018021528013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528013_C1/188,630,1069,1780/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528013_I1/48,140,469,380/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528014_C1/451,649,1327,1006/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528014_C3/489,2453,1352,382/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528014_I1/176,439,219,265/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528015_I1/1374,2644,214,202/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528016_I1/91,135,492,371/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528017_I1/165,341,216,262/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528019_I1/1001,872,208,235/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528020 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528020_C1/66,480,214,260/full/0/native.jpg

URN:NBN:no-nb_digibok_2018021528021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018021528021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018021528021_I1/93,369,82,176/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628002_I1/150,2264,214,222/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628003 



URN:NBN:no-nb_digibok_2018022628006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628006_I1/140,437,214,267/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628008 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628008_C1/162,298,206,178/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628010_C1/349,388,1038,1259/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628010_I1/50,469,218,262/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628011_I1/47,158,232,260/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628011_I3/90,153,1681,2840/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628012 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628012_I1/7,421,219,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628013_C1/606,1635,818,94/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628013_I1/919,1071,221,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628015_C1/429,1480,1245,924/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628015_I1/75,450,216,276/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628015_I1/810,913,652,1137/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628016_I1/64,126,399,402/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628017_C1/499,665,832,1301/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628017_I1/184,75,564,494/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628019 



URN:NBN:no-nb_digibok_2018022628020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628020 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628020_C1/816,2370,420,176/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628020_I1/139,402,214,248/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628021_C1/224,586,1723,2262/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628021_C3/259,1249,1810,1480/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628021_I1/194,455,218,257/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628022 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628022 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628022_I1/154,233,214,257/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628023 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628023 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628023_C1/1022,1467,832,1284/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628023_I1/120,496,232,306/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628024 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628024 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628024_I1/290,507,218,260/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628025 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628025 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628025_I1/0,0,1488,892/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628025_I3/30,2330,183,398/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628027 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628027 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628027_C1/578,1186,327,511/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628027_I1/0,2180,210,233/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628029 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628029 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628029_C3/102,2202,219,255/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628031 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628031 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628031_I1/104,344,216,257/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628032 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628032 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628032_C1/169,311,1768,1003/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628032_C1/205,2821,279,246/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628032_I1/61,37,464,436/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628033 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628033 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628033_I1/254,619,218,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628034 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628034 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628034_I1/110,262,214,281/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628035 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628035 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628035_C1/522,36,854,1083/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628035_I1/121,437,218,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628036 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628036 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628036_I1/132,383,214,232/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628037 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628037 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628037_C3/121,2074,1469,611/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628037_I1/34,255,406,333/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628038 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628038 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018022628038_I1/139,425,222,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2018022628039 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018022628039 



URN:NBN:no-nb_digibok_2018102326002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326002_C1/606,513,895,875/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326002_C3/889,268,505,423/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326002_I1/1545,192,339,344/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326003_C1/663,843,684,1053/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326003_I1/309,358,94,165/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326004_I1/184,455,218,276/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326006_C1/489,262,794,968/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326006_C3/237,1865,1224,799/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326006_I3/357,2444,218,259/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326007 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326007 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326007_C1/4,69,1731,1843/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326007_I1/1350,1264,213,229/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326008 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326008_C1/377,578,1256,1211/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326008_C3/191,402,214,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326009 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326009 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326009_I1/303,537,214,255/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326010_C1/549,586,717,1425/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326010_I1/1305,418,216,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326011_I1/206,363,219,243/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326013_C1/0,886,1845,1786/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326013_C1/0,2752,511,447/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326013_I1/270,401,218,232/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326014_C1/161,284,1545,2237/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326014_C1/154,2810,1551,244/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326014_I1/260,451,216,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326015_C3/206,2068,1262,311/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326016_C1/188,827,1738,1987/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326016_I1/205,448,219,237/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326016_I3/480,2679,1286,573/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326017_I1/304,399,218,244/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326019_C1/44,1162,1982,611/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326019_I1/327,413,216,267/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326020 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326020_C1/349,1074,1418,1684/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326020_I1/240,535,218,262/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326021_C1/31,350,1685,2578/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326021_I1/238,489,216,255/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326022 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326022 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326022_C1/18,1060,1848,1015/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326022_I1/251,409,221,257/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326026 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326026 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326026_I1/210,311,216,227/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326028 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326028 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326028_C3/447,2137,1237,374/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326028_I1/298,2539,549,300/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326029 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326029 



URN:NBN:no-nb_digibok_2018102326030 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326030 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326030_I1/1240,2518,213,225/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326032 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326032 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326032_I1/58,597,459,262/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326033 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326033 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326033_I1/306,524,216,241/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326033_I1/883,589,823,608/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326034 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326034 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326034_C1/23,481,1918,2659/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326035 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326035 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326035_I1/173,151,638,440/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326036 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326036 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326036_I1/183,206,469,292/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326037 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326037 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326037_C1/178,244,1692,2644/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326037_I1/298,589,148,219/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326039 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326039 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326039_C1/216,254,1387,1036/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326039_I1/1338,385,214,225/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326040 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326040 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326040_C1/466,884,1061,1358/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326040_I1/229,488,214,240/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326041 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326041 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326041_C1/342,826,1472,1233/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326041_I1/282,459,91,124/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326042 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326042 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326042_C1/462,922,1069,1000/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326042_I1/191,505,216,225/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326043 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326043 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326043_C3/186,2360,219,260/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326044 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326044 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326044_C3/227,2134,219,249/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326045 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326045 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326045_C1/178,480,1562,1308/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326045_I1/1532,284,211,233/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326046 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326046 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326046_C1/328,688,2005,1388/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326046_C3/1853,3060,262,314/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326046_I1/238,545,218,229/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326048 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326048 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326048_I1/113,80,459,360/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326049 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326049 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326049_C3/597,2970,1028,357/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326049_I1/259,391,214,284/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326050 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326050 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326050_I1/135,368,218,227/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326051 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326051 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326051_I1/191,470,216,224/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326052 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326052 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/745,413,148,284/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/1458,417,151,282/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/930,451,485,426/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/322,485,208,530/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/325,1489,137,431/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/1880,1491,151,641/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C1/319,2287,1831,1434/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_C3/158,1278,2412,1873/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_I1/210,541,357,255/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_I1/94,1034,821,2646/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326052_I1/1817,1592,780,2128/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326053 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326053 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326053_I1/259,116,459,443/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326054 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326054 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326054_C1/711,447,1074,376/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326054_C1/663,2366,843,366/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326054_I1/206,363,222,254/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326057 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326057 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326057_C3/1202,2333,218,255/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326058 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326058 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326058_I1/176,188,535,317/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326059 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326059 



URN:NBN:no-nb_digibok_2018102326061 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326061 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326061_C1/434,1156,1324,1538/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326061_C3/704,2535,1143,440/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326061_I1/301,485,216,227/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326062 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326062 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326062_C3/347,2003,1436,834/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326063 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326063 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326063_I1/216,357,213,227/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326064 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326064 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_C1/289,404,1361,1941/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_C1/289,2488,202,347/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_C3/557,728,270,210/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326064_I1/311,380,214,259/full/0/native.jpg

URN:NBN:no-nb_digibok_2018102326065 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018102326065 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326065_C3/669,1736,1447,1003/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018102326065_I1/292,474,216,263/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826003_C3/71,79,2120,2919/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826004_C3/235,1946,1733,973/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826004_I1/270,293,218,260/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826005 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826005 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826005_I1/135,273,486,412/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826006_C3/674,1047,1745,2087/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826006_I1/284,429,216,241/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826007 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826007 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826007_C3/527,1101,1442,1856/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826007_I1/273,447,214,249/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826009 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826009 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826009_I1/308,344,304,270/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826010_I1/142,248,413,404/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826010_I1/554,960,243,703/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826011_I1/229,319,211,243/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826011_I1/334,1131,243,720/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826012 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826012_I1/308,579,214,243/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826015_I1/184,276,213,146/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826016_C1/786,2115,279,142/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826017_C1/0,916,2079,910/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826017_I1/176,118,1956,1140/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826018 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826018 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826018_I1/93,109,1802,2303/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826019_C3/300,1926,2063,1218/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826019_I1/292,451,218,244/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826020 



URN:NBN:no-nb_digibok_2018112826021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826021_I1/252,214,540,303/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826024 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826024 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826024_C3/308,1649,1997,1259/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826024_I1/273,447,219,243/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826025 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826025 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826025_C1/0,124,1780,2891/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826025_I1/1523,432,218,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826027 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826027 



URN:NBN:no-nb_digibok_2018112826028 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826028 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826028_C3/132,1331,1908,1524/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826028_I1/184,401,218,237/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826028_I1/1772,2022,254,534/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826029 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826029 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826029_C1/973,2096,644,263/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826029_I1/148,146,357,407/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826030 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826030 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826030_C1/1328,2360,282,857/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826030_I1/507,418,218,232/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826033 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826033 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826033_C1/60,1068,1709,334/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826033_C3/77,79,1749,2453/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826033_I1/158,153,363,368/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826034 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826034 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826034_C1/943,474,575,521/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826036 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826036 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826036_C1/6,1393,2072,1346/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826036_I1/289,470,213,219/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826037 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826037 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826037_C1/707,1382,966,527/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826038 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826038 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C1/252,278,1369,1232/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C3/165,1376,764,737/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C3/1164,91,688,1173/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826038_C3/753,2467,281,189/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826039 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826039 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826039_I1/233,180,425,480/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826043 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826043 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826043_C1/522,1083,1074,597/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826043_I1/262,521,219,240/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826044 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826044 



URN:NBN:no-nb_digibok_2018112826045 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826045 



URN:NBN:no-nb_digibok_2018112826046 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826046 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_C1/17,510,1910,706/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_C1/18,1335,1944,2031/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_I1/992,173,1350,1140/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826046_I1/1763,1603,575,1088/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826047 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826047 



URN:NBN:no-nb_digibok_2018112826048 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826048 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826048_C3/1677,1303,71,432/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826048_C3/227,320,216,260/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826049 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826049 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826049_I1/290,312,211,213/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826050 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826050 



URN:NBN:no-nb_digibok_2018112826052 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826052 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826052_I1/353,464,90,164/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826054 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826054 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826054_C1/1695,94,447,470/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826054_I1/208,97,443,480/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826054_I3/1417,1889,797,1692/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826057 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826057 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_C1/436,1361,1298,951/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/206,388,210,218/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/1865,395,257,2954/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/466,1180,1363,1659/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826057_I3/213,3141,214,213/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826058 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826058 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826058_C1/459,1113,1320,1429/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826058_C3/88,75,2305,3599/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826058_I3/188,342,1927,3005/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826060 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826060 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826060_C1/475,1197,711,840/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826063 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826063 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826063_C1/184,453,131,1210/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826063_I1/214,172,404,480/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826064 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826064 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826064_C1/202,692,1598,1169/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826064_C3/232,2412,210,176/full/0/native.jpg

URN:NBN:no-nb_digibok_2018112826065 https://www.nb.no/items/URN:NBN:no-nb_digibok_2018112826065 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826065_C3/304,2215,1205,412/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2018112826065_I1/216,227,1156,257/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726008 



URN:NBN:no-nb_digibok_2019030726009 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726009 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726009_C1/652,782,1008,1564/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726010_C1/0,132,1518,1932/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726010_C3/1992,139,121,464/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726010_I1/290,554,213,216/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726011_C1/380,595,1450,2180/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726011_I1/1837,1112,74,451/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726011_I3/67,1150,74,418/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726012 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726012_C1/511,492,927,720/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726012_I3/669,2351,815,590/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726013 



URN:NBN:no-nb_digibok_2019030726014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726014_C1/556,709,750,701/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726014_I1/188,462,210,203/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726015_C1/347,815,1189,1017/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726015_I1/243,485,216,224/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726016_C1/0,1387,233,496/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726016_C3/132,1537,1458,1001/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726016_I3/273,1406,1215,1099/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726017_C1/425,1120,1027,1001/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726018 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726018 



URN:NBN:no-nb_digibok_2019030726021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726021_C1/619,1218,1181,1437/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726021_I1/304,641,218,249/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726022 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726022 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726022_C1/551,1142,1183,1434/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726023 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726023 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726023_C3/592,2613,1343,279/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726023_I1/379,453,218,229/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726024 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726024 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C1/331,194,1706,1085/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C1/26,1617,104,251/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C1/320,1628,1712,1561/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726024_C3/127,1617,2325,1373/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726025 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726025 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726025_0002/334,1211,638,188/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726025_0003/199,336,211,1412/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726025_C1/443,1366,1186,861/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726026 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726026 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726026_I1/488,358,1643,2782/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726026_I3/0,350,1831,2788/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726027 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726027 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726027_C3/643,1799,1504,910/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726028 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726028 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726028_C1/624,584,905,712/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726029 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726029 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726029_0001/142,142,1524,208/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726029_C3/699,2377,1038,341/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726030 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726030 



URN:NBN:no-nb_digibok_2019030726031 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726031 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726031_C3/331,1603,1096,554/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726032 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726032 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726032_I1/274,567,219,225/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726033 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726033 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726033_I1/1264,341,218,243/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726034 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726034 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726034_C3/1186,491,666,1959/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726034_I3/90,1726,1320,402/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726035 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726035 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726035_I1/216,221,192,200/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726036 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726036 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726036_C1/229,317,1676,2862/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726037 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726037 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726037_C1/53,2117,1832,756/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726037_I1/246,158,475,434/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726038 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726038 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_C1/515,676,624,317/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_C1/0,1439,146,393/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_C3/1518,2515,363,153/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726038_I1/229,453,216,224/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726039 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726039 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726039_I3/34,312,1606,1644/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726040 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726040 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726040_C1/34,1196,194,586/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726040_C1/361,771,1259,1448/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726040_I1/225,355,221,225/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726041 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726041 



URN:NBN:no-nb_digibok_2019030726042 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726042 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726042_C1/353,165,1428,590/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726042_I1/180,158,1485,690/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726043 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726043 



URN:NBN:no-nb_digibok_2019030726044 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726044 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726044_C1/557,1143,1227,1440/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726044_I1/213,202,382,412/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726045 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726045 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726045_C1/176,872,1477,2065/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726045_I1/268,235,936,447/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726046 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726046 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726046_C1/3,786,1774,2251/full/0/native.jpg

URN:NBN:no-nb_digibok_2019030726047 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019030726047 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019030726047_I1/235,298,213,221/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426002_C1/537,567,951,611/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426002_C3/257,485,214,230/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426003_I1/339,116,214,218/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426004_C1/323,606,1173,1257/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426004_I1/237,423,214,210/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426005 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426005 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426005_C1/304,692,1350,1267/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426005_I1/230,336,218,219/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426006_C1/537,930,976,1436/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426006_I1/333,497,214,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426007 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426007 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426007_C1/432,707,1222,1703/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426009 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426009 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426009_C1/764,639,687,929/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426010_C3/86,80,2164,3231/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426011_C3/462,2300,1185,598/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426011_I1/284,456,214,243/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426012 



URN:NBN:no-nb_digibok_2019031426013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426013_C3/165,1965,1278,679/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426013_I1/265,153,840,132/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426014_0002/206,2227,1467,522/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426014_C1/6,2000,1338,172/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0001/1180,60,311,276/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0002/1151,1344,499,891/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0002/121,2264,1523,409/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426015_0004/1107,90,576,2621/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426016_C1/66,451,1640,1422/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426016_I1/1518,110,409,458/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426017_C1/374,556,1347,1303/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426017_C3/1376,75,609,436/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426018 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426018 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426018_C1/317,658,1271,1617/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426018_I1/232,529,216,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426019_C1/176,303,1949,1358/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426019_I1/259,327,218,278/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426020 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426020_C1/230,118,1948,1376/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426020_I1/205,357,214,276/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426021_I1/251,355,214,274/full/0/native.jpg

URN:NBN:no-nb_digibok_2019031426022 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019031426022 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019031426022_I1/199,312,214,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326001 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326001 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326001_C1/31,104,1785,1279/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326001_I1/85,118,1543,560/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326002_C1/23,260,1865,2736/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326002_I1/175,158,423,464/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326002_I3/344,319,1564,753/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326003_C1/412,432,1295,1911/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326003_I1/165,158,1535,2719/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326004_C1/255,652,1685,1529/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326004_I1/170,180,1603,554/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326005 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326005 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326005_C1/377,121,1300,1518/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326005_I1/173,159,1990,2874/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326006_C1/328,720,1483,1641/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326006_I1/210,169,1641,511/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326007 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326007 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326007_C1/361,936,1286,1640/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326007_I1/270,331,216,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326008 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326008_C1/334,237,1502,1570/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326008_I1/216,150,1131,680/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326009 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326009 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326009_C1/311,545,1453,1622/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326009_I1/262,423,216,276/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326010_C1/120,364,1921,2747/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326010_I1/169,154,1373,429/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326011_C1/25,380,1529,1305/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326011_I1/127,165,1512,568/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_C1/0,692,1655,1452/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_C3/244,301,1872,2570/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_I1/222,172,1519,1227/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326013_I3/311,2123,1230,1003/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326014_C1/12,120,1918,1374/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326014_I1/191,126,1657,369/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326015_C1/15,790,1662,1545/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326015_I1/164,289,1355,532/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_C1/0,567,1987,2766/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_C3/1922,2908,279,440/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_I1/262,510,216,262/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_I1/668,812,1420,1243/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326016_I3/0,2910,244,439/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326017_C1/559,603,1009,1502/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326017_I1/153,402,214,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326018 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326018 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326018_C1/595,521,1039,1256/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326018_I1/252,284,205,271/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326019_C1/413,646,1331,1823/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326019_I1/159,112,1948,2904/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326020 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_C1/110,628,1712,1617/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_C3/90,339,1951,2867/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_I1/240,543,208,268/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326020_I3/758,309,761,2393/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326021_C1/96,139,1946,2987/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326021_I1/121,154,1478,532/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326022 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326022 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326022_C1/564,565,1047,1393/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326022_I1/344,102,1526,881/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326023 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326023 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326023_C1/336,741,1322,1412/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326023_I1/170,124,1594,1172/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326024 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326024 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326024_C1/469,594,1077,1194/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326024_I1/224,312,1463,1597/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326024_I1/1927,1388,181,243/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326025 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326025 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326025_C1/592,979,744,979/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326025_I1/206,105,1192,857/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326025_I3/75,154,1602,2344/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326026 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326026 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326026_C1/238,316,1549,2652/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326026_I1/137,153,524,472/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326027 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326027 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326027_I1/164,194,216,274/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326029 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326029 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326029_C1/369,869,1031,900/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326029_I1/158,113,1564,2918/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326030 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326030 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326030_I1/216,105,1308,376/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326031 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326031 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326031_C1/745,706,594,1019/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326031_I1/140,131,1417,488/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326038 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326038 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326038_C1/466,1305,785,666/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326038_I1/214,118,696,265/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326039 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326039 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326039_I1/341,387,213,273/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326040 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326040 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326040_C1/118,158,1238,2363/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326040_C3/246,2014,1426,603/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061326041 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061326041 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061326041_I1/336,120,1422,1221/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926001 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926001 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926001_I1/281,199,685,316/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926002_C1/176,218,1562,2633/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926002_C3/127,2242,1726,519/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926002_I1/391,180,712,331/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926003_I1/202,127,1621,736/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926004_C1/312,281,1670,2850/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926004_I1/158,143,214,262/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926005 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926005 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926005_C1/429,1093,1036,1382/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926005_I1/240,161,448,434/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926006_C1/429,744,1074,1203/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926006_I1/246,347,216,273/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926007 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926007 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926007_C1/60,557,1557,2096/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926007_I1/109,107,1589,952/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926008 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926008_I1/268,290,216,282/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926010_I1/270,402,214,274/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926010_I3/372,2406,1194,458/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926011_C1/320,949,1350,2472/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926011_C3/761,372,1109,2583/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926011_I1/260,235,372,461/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926012 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926012_C1/581,483,940,1431/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926012_I1/131,225,2133,1369/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926013_C1/379,733,1015,979/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926013_I1/154,284,214,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926014_C1/230,131,1420,2101/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926014_I1/120,270,1467,1118/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926015_I1/295,417,213,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926016_C1/1012,513,1377,2058/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926016_C1/421,2690,1979,862/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926016_I1/268,442,350,279/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926017_I3/333,2795,687,274/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926018 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926018 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926018_C1/537,1452,1077,824/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926018_C3/97,109,1916,3210/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926018_I3/355,339,1496,2689/full/0/native.jpg

URN:NBN:no-nb_digibok_2019061926019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019061926019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019061926019_I1/194,244,218,276/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926002 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926002 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926002_C3/1154,1944,349,458/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926003 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926003 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926003_C1/188,453,1633,1066/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926003_I1/334,327,1425,1058/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926004 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926004 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926004_I1/1635,504,202,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926005 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926005 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926005_C3/262,1202,1064,1011/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926005_I3/42,1354,1475,1055/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926006 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926006 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926006_C3/314,2711,1448,613/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926007 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926007 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926007_C1/232,1060,1401,1908/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926007_C3/314,477,211,268/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926008 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926008 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926008_C1/94,317,1820,2739/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926008_I1/273,353,213,246/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926008_I3/423,1884,1464,1150/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926009 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926009 



URN:NBN:no-nb_digibok_2019112926010 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926010 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926010_C1/334,658,1082,771/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926010_I1/322,183,1665,1385/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926011 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926011 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926011_C1/172,554,1549,2496/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926012 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926012 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926012_I1/227,229,1627,693/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926013 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926013 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926013_C1/20,251,1941,2872/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926014 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926014 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_C1/421,682,1333,1188/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_C3/467,383,1248,2725/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_I1/213,237,1810,1452/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926014_I3/274,1598,1775,1488/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926015 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926015 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_C1/58,532,1810,2837/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_C3/235,541,2008,2548/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_I1/417,526,1395,2545/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926015_I3/474,541,1134,2362/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926016 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926016 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_C1/311,695,1619,2640/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_C3/1543,669,366,929/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_I1/227,374,1813,2968/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926016_I3/347,366,1704,2918/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926017 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926017 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926017_C1/230,1205,1896,1971/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926018 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926018 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926018_C1/666,794,774,756/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926019 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926019 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926019_C1/222,748,1761,2388/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926019_C3/290,260,188,251/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926020 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926020 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926020_C3/304,317,1549,2866/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926021 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926021 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926021_C1/33,407,1949,2776/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926021_I1/221,227,1715,1591/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926022 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926022 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926022_C1/0,210,1777,2994/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926022_I1/325,458,203,293/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926022_I1/2022,364,150,131/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926023 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926023 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926023_C1/44,229,1967,2302/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926023_C3/276,1946,1616,1094/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926023_I1/338,301,1346,2690/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926024 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926024 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926024_C1/192,347,1804,2084/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926024_I1/361,388,200,259/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926025 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926025 



URN:NBN:no-nb_digibok_2019112926026 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926026 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926026_C1/235,232,1747,2888/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926026_I1/314,214,1546,2700/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926027 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926027 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_C1/45,137,1973,3049/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_C3/205,123,1926,3049/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_I1/165,248,1940,2674/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926027_I3/0,287,1829,2674/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926028 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926028 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926028_C1/385,706,1319,941/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926029 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926029 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926029_C1/184,175,1771,2943/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926029_I1/188,178,613,249/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926030 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926030 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926030_C1/213,521,1839,1813/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926030_I1/110,121,1523,1210/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926031 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926031 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926031_C1/317,227,1572,2635/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926031_C3/72,120,364,398/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926032 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926032 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926032_I1/99,121,1911,3038/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926032_I3/37,1301,1780,1734/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926033 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926033 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_C1/638,218,1102,191/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_C1/126,780,1804,2584/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_C3/235,445,1717,2335/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_I1/301,235,1494,339/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926033_I1/284,1377,1791,1064/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926034 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926034 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926034_C1/143,202,1747,3117/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926034_C3/281,252,1646,3028/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926034_I3/350,2904,753,297/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926035 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926035 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_C1/372,861,1088,1075/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_C3/282,499,1564,2050/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_I1/265,218,1409,519/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_I1/480,2682,1205,553/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926035_I3/333,483,1287,2379/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926036 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926036 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926036_C1/295,240,1619,3139/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926036_I1/273,227,1668,2913/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926037 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926037 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926037_C3/257,496,205,259/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926038 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926038 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926038_C1/186,189,1665,3036/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926038_C3/421,2788,211,267/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926039 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926039 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926039_C1/213,451,1788,2923/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926039_C3/320,2376,1519,1019/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926040 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926040 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926040_C3/216,355,205,251/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926040_C3/246,1488,1712,1722/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926041 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926041 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926041_C3/262,238,1636,3148/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926042 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926042 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926042_I1/282,543,213,262/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926043 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926043 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926043_C3/251,333,1151,464/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926044 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926044 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926044_C3/219,395,189,252/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926045 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926045 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926045_C1/227,206,1744,2853/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926045_C3/210,319,1636,2801/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926046 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926046 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926046_C1/154,200,1902,3082/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926046_C3/276,2690,1580,350/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926047 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926047 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926047_C1/244,216,1548,2945/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926047_C3/248,2632,189,526/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926048 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926048 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926048_C1/1741,278,282,186/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926048_C1/162,1611,1712,1545/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926048_C3/170,214,1862,3050/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926049 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926049 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926049_C1/246,203,1831,3117/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926049_C3/363,229,1676,2885/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926050 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926050 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926050_C1/176,380,1655,2234/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926050_C3/260,385,243,267/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926051 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926051 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926051_C1/361,221,1616,2900/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926051_C3/176,401,1899,2450/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926052 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926052 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926052_C1/146,197,1744,2859/full/0/native.jpg

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926052_C3/229,339,1625,2611/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926053 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926053 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926053_C1/178,676,1580,1085/full/0/native.jpg

URN:NBN:no-nb_digibok_2019112926054 https://www.nb.no/items/URN:NBN:no-nb_digibok_2019112926054 

https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2019112926054_I1/428,2720,1445,374/full/0/native.jpg

Lagrer skillingsvisene til fil

In [160]:
import json
json.dump(skillingsbilder, open('skillingsbilder.json','w', encoding = 'utf-8'))

Se på bilder

In [165]:
vis_bok = lambda x: HTML(display_books({u:skillingsbilder[u] for u in skillingsbilder if x in u}))

En hjelpekommando for å vise enkeltbøker

In [167]:
vis_bok("nb_digibok_2018112826057")

In [162]:
html_bilder = display_books(dict_sample(skillingsbilder), 400)

In [163]:
HTML(html_bilder)